In [1]:
import requests,json
import base64
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import cv2


url = 'https://h5.ele.me/restapi/eus/login/mobile_send_code'
session = requests.session()
#创建会话。
headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
}
#payload = {'mobile': '18810607138', 'captcha_value': "", 'captcha_hash': "", 'scf': "ms"}
#res = session.post(url, headers=headers, data=payload)
#print(res.json())

In [17]:
def Token_img_show():
    urlTokenImg = 'https://h5.ele.me/restapi/eus/v3/captchas'
    dataTokenImg = {
        "captcha_str":'18810607138'
        }
    tokenImgDatas = session.post(urlTokenImg,headers=headers,data=dataTokenImg)
    tokenImg64Code = tokenImgDatas.json()['captcha_image']#取得验证码的base64编码 另一键：captcha_hash
        #【上面注意json转字典.json()】
        #print(tokenImg64Code)
        #data:image/jpeg;base64,/9j/4AAQ......
        #testBase64 = '/9j/4AAQSkZJRg......'
        #【注意：试存静态编码转base64可知 上面base64编码要split一下，逗号后面是】
    #rint(tokenImg64Code)
    tokenImg64Code = tokenImg64Code.split(',')[1]#索引是从左向右0，1，2这样的注意
    #rint(tokenImg64Code)
    tokenImg = base64.b64decode(tokenImg64Code)#转成图片存到本地
    with open(r'token.jpg','wb+') as fw:
        fw.write(tokenImg)
    plt.imshow(cv2.imread('token.jpg'))
    plt.show()
 #==取验证码，取完图片验证码
    # 根据本地图片输入，传到下面取手机验证码（比没图片验证码时多了一步，原理是一样的）=
    
def Post_token_img():
    urlToken = 'https://h5.ele.me/restapi/eus/login/mobile_send_code'
    tokenImgValue = input('请输入图片验证码，图片见本机已下载：')
    #请求带有验证码的 手机验证码发送
    dataToken = {
        "captcha_hash":"",
        "captcha_value":tokenImgValue,
        "mobile":'18810607138',
        "scf":"ms"
    }
    tokenTel = session.post(urlToken,headers=headers,data=dataToken)#只有一键validate_token
    print(tokenTel.json())
    return tokenTel

def login(tokenTel):
    validateToken = tokenTel.json()['validate_token']
    #验证码传入登录请求
    validateCode = input('请输入手机验证码：')
    #手机输入验证码，传入登录请求
    #上面手机验证的validateToken validateCode两个一起传入登录请求中
    #==登录==
    urlLogin = 'https://h5.ele.me/restapi/eus/login/login_by_mobile'
    #登录请求是怎么找到的？页面正常登录勾选network里Preserve log,完成登录后即可找到。Preserve log可保障不会登录后跳走消息这个请求
    dataLogin = {
        "mobile":'18810607138',
        "scf":"ms",
        "validate_code":validateCode,
        "validate_token":validateToken
    }
    login_res = session.post(urlLogin,headers=headers,data=dataLogin)
    if login_res.status_code ==200:
        print('登陆成功')
    else :
        print('登陆失败')
    return login_res

def save_cookies():
    cookies_dict = requests.utils.dict_from_cookiejar(session.cookies)
    cookies_str = json.dumps(cookies_dict)
    with open('cookies_eleme.txt', 'w') as f:
        f.write(cookies_str)
def cookies_read():
    cookies_txt = open('cookies_eleme.txt', 'r')
    cookies_dict = json.loads(cookies_txt.read())
    cookies = requests.utils.cookiejar_from_dict(cookies_dict)
    return (cookies)
    # 以上4行代码，是cookies读取。
def serch_resturnts(batch):
    url_base = 'https://h5.ele.me/restapi/shopping/v3/restaurants?'
    for i in range(batch):
        offset = i*8
        if i ==0:
            rank_id = ''
        else:
            rank_id = '34972c6dc3214fbd871ab90d11d3b6c6'
        params = {
            'latitude': '39.990348',
            'longitude': '116.359323',
            'offset': str(offset),
            'limit': '8',
            'extras[]': 'activities',
            'extras[]': 'tags',
            'extra_filters': 'home',
            'rank_id' : rank_id, 
            'terminal': 'h5'
        }
        resturant = session.get(url_base,headers = headers,params=params)
        print(resturant.status_code)
        if resturant.status_code ==200:
            resturants_list = resturant.json()['items']
            for i in range(len(resturants_list)):
                print(resturants_list[i]['restaurant']['name'])
        else:
            print('cookies过期了，请重新登陆')
            Post_token_img()
            #针对饿了么的反爬技术我们先把验证码图片存到本地
            Token_img_show()
            #看到验证码后手动输入，（这里可以介入一个验证码识别模块，直接识别输入，就没必要自己手动输入了）
            tokenTel = Post_token_img() #手动输入验证码后，才能发出短信
            login(tokenTel)#登陆
            save_cookies()
            
def cookies_login():
    try :
        cookies = cookies_read()
        print('读取本地cookies成功') #如果不是手机验证码的话，是可以不用重复登陆的
        session.cookies = cookies
    except FileNotFoundError:
        print('正在重新登陆')
        Post_token_img()
        #针对饿了么的反爬技术我们先把验证码图片存到本地
        Token_img_show()
    #看到验证码后手动输入，（这里可以介入一个验证码识别模块，直接识别输入，就没必要自己手动输入了）
        tokenTel = Post_token_img() #手动输入验证码后，才能发出短信
        login(tokenTel)#登陆
        save_cookies()
    except :
        print('不能cookies自动登陆')
        pass

def main():
    url = 'https://h5.ele.me/restapi/eus/login/mobile_send_code'
    session = requests.session()
    #创建会话。
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
    }
    cookies_login()
    batch_number = int(input('输入你想爬取的店铺匹数，8个为一批'))
    serch_resturnts(batch_number)

In [18]:
if __name__ == '__main__':
    main()

读取本地cookies成功
输入你想爬取的店铺匹数，8个为一批30
200
共禾之约·创意轻食(第B8档口五道口熊猫星厨美食城店)
呷哺呷煮呷烫(翠微牡丹园店)
巨饿便当(中关村东路店)
台资味·卤肉饭(北京六道口店)
麻辣鸡架(第3档口燕京东餐饮美食店)
北京麦当劳志新路餐厅
一虾一刻小龙虾饭·料理(中关村东路店)
京味斋(花园路店)
200
壹只蟹蟹煲饭(第3档口生鲜美食城店)
麻辣鸡架(第3档口燕京东餐饮美食店)
爱村鱼籽拌饭(学院路店)
曼玲粥店(六道口店)
大东北小锅饭(第20号档口香四溢美食城店)
小家韩式餐吧(农大店)
渝小碗(第3号档口西奥美食城店)
金百万烤鸭(牡丹园店)
200
果多美(北京花园路店)
川妹儿(六道口店)
田老师红烧肉(建德门店)
驴肉火烧(旗舰店)
紫燕百味鸡(花园东路店)
米多面多(第4档口亿特美食城店)
肯德基宅急送(学院路店）
梁小猴铁板炒饭(第8号档口花园美食城店)
200
舌尖滋味麻辣烫·麻辣拌(第29号档口花园美食城店)
超市发（学院路店）
张姐脆皮鸡烤肉饭(第5档口知春食佳美食城店)
卤掌门卤肉饭(六道口店)
河北名吃驴肉火烧(农大店)
米先生黯然销魂饭（五道口店）
杨铭宇黄焖鸡米饭(农大店)
吉野家(二里庄店)
200
付小姐在成都(六道口店)
西贝莜面村（北京五道口店）
焖范儿·三汁焖饭(北京六道口店)
必胜客宅急送（花园路店）
原味觉醒健康餐（五道口店）
汉堡王(五道口店19195)
渝是乎(牡丹园店)
西少爷肉夹馍(五道口店)
200
南京大牌档（中关村店）
二十五块半·卤掌门（牡丹园店）
小恒水饺(第3号档口利康金桥美食城店)
德克士（五道口店）
好嫂子(牡丹园店)
池记串吧(北影店)
局气（五道口店）
火齐潮汕砂锅粥(牡丹园店)
200
刀小蛮半只鸡云南米线(五道口华联店)
东方饺子王（健德桥店）
鲜一汤饭(香四溢美食城14号档口店)
火炉火韩式拌饭(五道口店)
大鸭梨烤鸭(亚运村店)
萨莉亚(25北京学清路店)
港岛记·香港叉烧饭·知春路(第15号档口双星艺洁美食中心店)
麦多馅饼(知春路店)
200
四有青年米粉·面(五道口店)
三味口岸麻辣拌(五道口店)
日昌餐馆(五道口店)
百万庄园西餐（牡丹园店）
庆丰包子铺(志新路店)
金掌勺（双榆树店）
凯哥猪脚饭(第6号档口金福盛悦美食城店)
匠骨头(第